# 0. Librerias

In [2]:
# Genérica
# -----------------------------------------------------------------------------
from time import time
import session_info


# Tratamientos datos
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np


# category encoders
# -----------------------------------------------------------------------------
from category_encoders import OrdinalEncoder


# scikit-learn
# -----------------------------------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# LightGBM
# -----------------------------------------------------------------------------
import lightgbm as lgb


# XGBoost
# -----------------------------------------------------------------------------
from xgboost import XGBRegressor


# Gráficos
# -----------------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt


session_info.show(html=False)

-----
category_encoders   2.6.1
lightgbm            3.3.5
matplotlib          3.7.2
numpy               1.23.5
pandas              1.5.3
seaborn             0.12.2
session_info        1.0.0
sklearn             1.3.0
xgboost             1.7.6
-----
IPython             8.14.0
jupyter_client      8.3.0
jupyter_core        5.3.1
notebook            6.5.4
-----
Python 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
Linux-5.15.0-91-generic-x86_64-with-glibc2.31
-----
Session information updated at 2023-12-19 11:10


In [3]:
# Funciones
# -----------------------------------------------------------------------------
def optimize_df_dtypes(df):

    variables_categoricas = [
        'Epoca_Covid',
        'Anio',
        'Trimestre',
        'Mes',
        'Cliente',
        'Tipo_Cliente',
        'Distribuidor_MSM',
        'Codigo_Postal',
        'Area_Dist',
        'Zona',
        'ID_CCAA',
        'Comunidad_Autonoma',
        'ID_Provincia',
        'Provincia',
        'Gama',
        'Formato_1',
        'Formato_2'
        ]

    for i in df.columns:
        if i in variables_categoricas:
            df[i] = df[i].astype("category")
 
    fcols = df.select_dtypes('float').columns
    icols = df.select_dtypes('integer').columns

    df[fcols] = df[fcols].apply(pd.to_numeric, downcast='float')
    df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')

    return df 

# 1. Cargar datos

In [13]:
# Ruta
path = './data/tablon-sell-in-4.csv'

# Cargar datos
df = pd.read_csv(path, sep=';')

# Optimizar tipo dato
df = optimize_df_dtypes(df)

# Eliminar año 2020
filtro = (df.Anio != '2020')
df = df[filtro]

df.head()

/tmp/ipykernel_30759/2961699486.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=';')


,Fecha,Epoca_Covid,Anio,Trimestre,Mes,Cliente,Tipo_Cliente,Distribuidor_MSM,Codigo_Postal,Area_Dist,...,IPRI,Temperatura_Minima,Temperatura_Media,Temperatura_Maxima,Precipitacion,Sell_In_3,Sell_In_2,Sell_In_1,Sell_In,STRATIO_CREATED_DATE
0,201610,pre-covid,2016,T4,10,534564,DHC,0,30004,ZZ,...,102.449997,15.11,20.379999,25.66,27.65,0.32,0.32,0.32,0.32,2023-12-14T16:00:48.156Z
1,201610,pre-covid,2016,T4,10,534564,DHC,0,30004,ZZ,...,102.449997,15.11,20.379999,25.66,27.65,0.00,0.00,0.00,0.00,2023-12-14T16:00:48.156Z
2,201610,pre-covid,2016,T4,10,534564,DHC,0,30004,ZZ,...,102.449997,15.11,20.379999,25.66,27.65,3.00,3.00,3.00,3.00,2023-12-14T16:00:48.156Z
3,201610,pre-covid,2016,T4,10,534564,DHC,0,30004,ZZ,...,102.449997,15.11,20.379999,25.66,27.65,0.06,0.06,0.06,0.06,2023-12-14T16:00:48.156Z
4,201610,pre-covid,2016,T4,10,534564,DHC,0,30004,ZZ,...,102.449997,15.11,20.379999,25.66,27.65,0.16,0.16,0.16,0.16,2023-12-14T16:00:48.156Z


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2644111 entries, 0 to 2644110
Data columns (total 58 columns):
 #   Column                      Dtype   
---  ------                      -----   
 0   Fecha                       int32   
 1   Epoca_Covid                 category
 2   Anio                        category
 3   Trimestre                   category
 4   Mes                         category
 5   Cliente                     category
 6   Tipo_Cliente                category
 7   Distribuidor_MSM            category
 8   Codigo_Postal               category
 9   Area_Dist                   category
 10  Zona                        category
 11  ID_Provincia                category
 12  Provincia                   category
 13  ID_CCAA                     category
 14  Comunidad_Autonoma          category
 15  Gama                        category
 16  Formato_1                   category
 17  Formato_2                   category
 18  Numerica                    float32 
 19  

In [15]:
# Eliminar columnas genéricas para todos los experimentos
df = df.drop(columns=[
    'Fecha',
    'Anio',
    'Cliente',
    'Codigo_Postal',
    'Provincia',
    'Comunidad_Autonoma',
    'Frontur',
    'Temperatura_Minima',
    'Temperatura_Maxima',
    'Movimientos_Entrada',
    'Movimientos_Salida',
    'Volumen_Entregado_Perfecto',
    'Numerica',
    'Sell_In_3',
    'Sell_In_2',
    'STRATIO_CREATED_DATE'
])

# Eliminar registros con NaN
df = df.dropna()

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2644111 entries, 0 to 2644110
Data columns (total 42 columns):
 #   Column               Dtype   
---  ------               -----   
 0   Epoca_Covid          category
 1   Trimestre            category
 2   Mes                  category
 3   Tipo_Cliente         category
 4   Distribuidor_MSM     category
 5   Area_Dist            category
 6   Zona                 category
 7   ID_Provincia         category
 8   ID_CCAA              category
 9   Gama                 category
 10  Formato_1            category
 11  Formato_2            category
 12  Num_PdV_CI           float32 
 13  Num_PdV_RU           float32 
 14  Num_PdV_TU           float32 
 15  Impactos_TV          float64 
 16  Impactos_OOH         float64 
 17  Impresiones_Meta     float32 
 18  Impresiones_Pint     float32 
 19  Impresiones_Digi     float32 
 20  Sell_In_PP           float64 
 21  Marketing_Nacional   float64 
 22  Trade_Marketing      float64 
 23  Marketi

In [17]:
# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df.columns)
features.remove(target)

# Variables categóricas
categorical_features = [col for col in df.columns if df[col].dtype == 'category']

# Mostrar
print(f'Variable dependiente: {target}')
print(30*'-')
print(f'Variables independientes: {features}')
print(30*'-')
print(f'Variables categóricas: {categorical_features}')

Variable dependiente: Sell_In
------------------------------
Variables independientes: ['Epoca_Covid', 'Trimestre', 'Mes', 'Tipo_Cliente', 'Distribuidor_MSM', 'Area_Dist', 'Zona', 'ID_Provincia', 'ID_CCAA', 'Gama', 'Formato_1', 'Formato_2', 'Num_PdV_CI', 'Num_PdV_RU', 'Num_PdV_TU', 'Impactos_TV', 'Impactos_OOH', 'Impresiones_Meta', 'Impresiones_Pint', 'Impresiones_Digi', 'Sell_In_PP', 'Marketing_Nacional', 'Trade_Marketing', 'Marketing_PLV', 'Distancia_CdP', 'Dispersion_PdV', 'Volumen_Solicitado', 'Nivel_Servicio', 'Tarifa_Eur_Hl', 'Margen_Distribuidor', 'Num_Festivos', 'Num_Festivos_AI', 'Egatur', 'Familitur', 'Movimientos_Neto', 'IPC', 'IPI', 'IPRI', 'Temperatura_Media', 'Precipitacion', 'Sell_In_1']
------------------------------
Variables categóricas: ['Epoca_Covid', 'Trimestre', 'Mes', 'Tipo_Cliente', 'Distribuidor_MSM', 'Area_Dist', 'Zona', 'ID_Provincia', 'ID_CCAA', 'Gama', 'Formato_1', 'Formato_2']


# 2. MLflow

In [10]:
# Importar MLflow
import mlflow
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema, ColSpec

In [12]:
# Ver ruta actual desde donde se va a levantar el MLflow server
!pwd

/home/gmachin/git/personal/msm_sell_in_prediction


In [10]:
# Iniciar un servidor de MLflow con alguno de los dos comandos siguientes en una terminal
# $ mlflow server --host 127.0.0.1 --port 5000
# $ mlflow ui

In [11]:
# Apuntar el notebook al servidor MLflow levantado en la celda anterior
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [21]:
# Crear un experimento
# --------------------------------------------------

# Nombre
experiment_name = "MSM_Sell_In_Explicabilidad"

# Descripción
experiment_description = (
    "Proyecto de explicabilidad de Sell In."
)

# Etiquetas del experimento
experiment_tags = {
    "project_name": "Explicabilidad-Sell-In",
    "problem_type": "Regresión",
    "client": "MSM",
    "mlflow.note.content": experiment_description,
}

# Crear el experimento
produce_sell_in_experiment = mlflow.create_experiment(
    name=experiment_name,
    tags=experiment_tags
)

# Activar el experimento
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='mlflow-artifacts:/543627291029851046', creation_time=1701166428072, experiment_id='543627291029851046', last_update_time=1701166428072, lifecycle_stage='active', name='MSM_Sell_In_Explicabilidad', tags={'client': 'MSM',
 'mlflow.note.content': 'Proyecto de explicabilidad de Sell In.',
 'problem_type': 'Regresión',
 'project_name': 'Explicabilidad-Sell-In'}>

# 3. Modelos

## 3.1. LightGBM (Todas las variables)

In [22]:
# Medir tiempo
tiempo_inicial = time()


# Variables de ejecución MLflow
# ------------------------------------------------------------------------------------
experiment_name = 'MSM_Sell_In_Explicabilidad'
run_name = 'Modelo_XAI_LightGBM_All_Variables'
model_name = 'lightgbm-model-all-var'


# Seed
# ------------------------------------------------------------------------------------
seed = 123


# Dataset de test y entrenamiento
# ------------------------------------------------------------------------------------
# Copia del dataset
df_exp = df.copy()

# Eliminar variables con menor infulencia en el modelo (sobre 100)
# ...
# ...

# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df_exp.columns)
features.remove(target)

# Variables categóricas
categorical_features = [col for col in df_exp.columns if df_exp[col].dtype == 'category']

# Dataset de test y entrenamiento
train, test = train_test_split(df_exp, train_size=0.95, test_size=0.05, random_state=seed)


# MLflow
# ------------------------------------------------------------------------------------
# Activar el experimento
mlflow.set_experiment(experiment_name)

# Lanzar ejecución de MLflow
with mlflow.start_run(run_name=run_name) as run:
    
    # UUID del experimento
    experiment_id = run.info.experiment_id
    
    # UUID de la ejecución
    run_id = run.info.run_uuid
    
    # Instanciar OrdinalEncoder
    encoder = OrdinalEncoder(
        cols=categorical_features,
        handle_unknown="ignore",
        return_df=True
    )

    
    # Parámetros
    params = {
        "boosting_type": "gbdt",
        "learning_rate": 0.1,
        "n_estimators": 1000,
        "max_depth": 8,
        "num_leaves": 120,
        "min_child_samples": 10,
        "bagging_fraction": 1.0,
        "bagging_freq": 0,
        "feature_fraction": 0.8,
        "reg_sqrt": True,
        "feature_pre_filter": False,
        "random_state": seed,
        "n_jobs": -1
    }
    

    # Instanciar LGBMRegressor
    mlgb = lgb.LGBMRegressor(**params)

    
    # Pipeline: Encoder + Regresor
    model_pipeline = Pipeline([
        ("encoder", encoder),
        ("mlgb", mlgb)
    ])
    
    
    # Entrenar modelo
    model = model_pipeline.fit(train[features], train[target])
    
    
    # Cálculo de métricas
    train_y_real = train[target]
    train_y_pred = model.predict(train[features])

    test_y_real = test[target]
    test_y_pred = model.predict(test[features])

    metric_r2_train = round(r2_score(train_y_real, train_y_pred), 3)
    metric_r2_test = round(r2_score(test_y_real, test_y_pred), 3)

    metric_r2_adj_train = round(1-(1-metric_r2_train)*(len(test)-1)/(len(test)-len(features)-1), 3)
    metric_r2_adj_test = round(1-(1-metric_r2_test)*(len(test)-1)/(len(test)-len(features)-1), 3)

    metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
    metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

    metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
    metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

    metric_train_score = round(1.00 - (metric_train_mae / train_y_real.mean()), 3)
    metric_test_score = round(1.00 - (metric_test_mae / test_y_real.mean()), 3)
    
    
    # Genere un diccionario con las métricas
    metrics = {
        "r2_train": metric_r2_train,
        "r2_test": metric_r2_test,
        "r2_adj_train": metric_r2_adj_train,
        "r2_adj_test": metric_r2_adj_test,
        "mae_train": metric_train_mae,
        "mae_test": metric_test_mae,
        "rmse_train": metric_train_rmse,
        "rmse_test": metric_test_rmse,
        "score_train": metric_train_score,
        "score_test": metric_test_score
    }
    
    
    # Loggar parámetros
    mlflow.log_params(params)
    
    # Logar métricas
    mlflow.log_metrics(metrics)
    
    # Schema input/output del modelo
    input_schema = Schema([
        ColSpec("string", "Epoca_Covid"),
        ColSpec("string", "Trimestre"),
        ColSpec("string", "Mes"),
        ColSpec("string", "Tipo_Cliente"),
        ColSpec("string", "Distribuidor_MSM"),
        ColSpec("string", "Area_Dist"),
        ColSpec("string", "Zona"),
        ColSpec("string", "ID_CCAA"),
        ColSpec("string", "ID_Provincia"),
        ColSpec("string", "Gama"),
        ColSpec("string", "Formato_1"),
        ColSpec("string", "Formato_2"),
        ColSpec("double", "Num_PdV_CI"),
        ColSpec("double", "Num_PdV_RU"),
        ColSpec("double", "Num_PdV_TU"),
        ColSpec("double", "Impactos_TV"),
        ColSpec("double", "Impactos_OOH"),
        ColSpec("double", "Impresiones_Meta"),
        ColSpec("double", "Impresiones_Pint"),
        ColSpec("double", "Impresiones_Digi"),
        ColSpec("double", "Sell_In_PP"),
        ColSpec("double", "Marketing_Nacional"),
        ColSpec("double", "Trade_Marketing"),
        ColSpec("double", "Marketing_PLV"),
        ColSpec("double", "Distancia_CdP"),
        ColSpec("double", "Dispersion_PdV"),
        ColSpec("double", "Volumen_Solicitado"),
        ColSpec("double", "Nivel_Servicio"),
        ColSpec("double", "Tarifa_Eur_Hl"),
        ColSpec("double", "Margen_Distribuidor"),
        ColSpec("double", "Num_Festivos"),
        ColSpec("integer", "Num_Festivos_AI"),
        ColSpec("double", "Egatur"),
        ColSpec("double", "Familitur"),
        ColSpec("double", "Movimientos_Neto"),
        ColSpec("double", "IPC"),
        ColSpec("double", "IPI"),
        ColSpec("double", "IPRI"),
        ColSpec("double", "Temperatura_Media"),
        ColSpec("double", "Precipitacion"),
        ColSpec("double", "Sell_In_1")
    ])
    
    output_schema = Schema([ColSpec("double", "prediction")])
    
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    # Log modelo entrenado
    mlflow.sklearn.log_model(model, model_name, signature=signature)

    
    
    
# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f"El tiempo de ejecución es de {tiempo_ejecucion/60} minutos")


# Info de la ejecución
print("")
print(f"Modelo loggado en el experimento: {experiment_id} --> {experiment_name}")
print(f"Modelo loggado en la ejecución: {run_id} --> {run_name}")
print(f"Nombre del modelo: {model_name}")
print(f"Modelo URI: 'runs:/{run_id}/artifacts/{model_name}'")

[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0

El tiempo de ejecución es de 2.6361494064331055 minutos

Modelo loggado en el experimento: 543627291029851046 --> MSM_Sell_In_Explicabilidad
Modelo loggado en la ejecución: 765d2f6fd9b44b878d673aba36da5d6d --> Modelo_XAI_LightGBM_All_Variables
Nombre del modelo: lightgbm-model-all-var
Modelo URI: 'runs:/765d2f6fd9b44b878d673aba36da5d6d/artifacts/lightgbm-model-all-var'


/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## 3.2. LightGBM (Reducción Variables - Experimento 1)

Ejecución probando a eliminar las variables que tienen menos importancia en el modelo:

In [23]:
# Medir tiempo
tiempo_inicial = time()


# Variables de ejecución MLflow
# ------------------------------------------------------------------------------------
experiment_name = 'MSM_Sell_In_Explicabilidad'
run_name = 'Modelo_XAI_LightGBM_Experiment_1'
model_name = 'lightgbm-model-exp-1'


# Seed
# ------------------------------------------------------------------------------------
seed = 123


# Dataset de test y entrenamiento
# ------------------------------------------------------------------------------------
# Copia del dataset
df_exp = df.copy()

# Eliminar variables con menor infulencia en el modelo (sobre 100)
df_exp = df_exp.drop(columns=[
    'Num_Festivos_AI',      # 0.00
    'Distribuidor_MSM',     # 0.01
    'Impactos_OOH',         # 0.01
    'Impresiones_Digi',     # 0.04
    'Impresiones_Pint',     # 0.06
    'Trimestre',            # 0.09
    'Num_Festivos',         # 0.11
    'ID_CCAA',              # 0.11
    'Nivel_Servicio',       # 0.12
    'Zona',                 # 0.21
    'Precipitacion',        # 0.23
    #'Num_PdV_RU',          # 0.29
    'Temperatura_Media',    # 0.31
    #'Num_PdV_TU',          # 0.33
    'Distancia_CdP',        # 0.33
    'Movimientos_Neto',     # 0.35
    'IPI',                  # 0.41
    'Impresiones_Meta',     # 0.42
    'IPC'                   # 0.49
])

# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df_exp.columns)
features.remove(target)

# Variables categóricas
categorical_features = [col for col in df_exp.columns if df_exp[col].dtype == 'category']

# Dataset de test y entrenamiento
train, test = train_test_split(df_exp, train_size=0.95, test_size=0.05, random_state=seed)



# MLflow
# ------------------------------------------------------------------------------------
# Activar el experimento
mlflow.set_experiment(experiment_name)

# Lanzar ejecución de MLflow
with mlflow.start_run(run_name=run_name) as run:
    
    # UUID del experimento
    experiment_id = run.info.experiment_id
    
    # UUID de la ejecución
    run_id = run.info.run_uuid
    
    # Instanciar OrdinalEncoder
    encoder = OrdinalEncoder(
        cols=categorical_features,
        handle_unknown="ignore",
        return_df=True
    )

    
    # Parámetros
    params = {
        "boosting_type": "gbdt",
        "learning_rate": 0.1,
        "n_estimators": 1000,
        "max_depth": 8,
        "num_leaves": 120,
        "min_child_samples": 10,
        "bagging_fraction": 1.0,
        "bagging_freq": 0,
        "feature_fraction": 0.8,
        "reg_sqrt": True,
        "feature_pre_filter": False,
        "random_state": seed,
        "n_jobs": -1
    }
    

    # Instanciar LGBMRegressor
    mlgb = lgb.LGBMRegressor(**params)

    
    # Pipeline: Encoder + Regresor
    model_pipeline = Pipeline([
        ("encoder", encoder),
        ("mlgb", mlgb)
    ])
    
    
    # Entrenar modelo
    model = model_pipeline.fit(train[features], train[target])
    
    
    # Cálculo de métricas
    train_y_real = train[target]
    train_y_pred = model.predict(train[features])

    test_y_real = test[target]
    test_y_pred = model.predict(test[features])

    metric_r2_train = round(r2_score(train_y_real, train_y_pred), 3)
    metric_r2_test = round(r2_score(test_y_real, test_y_pred), 3)

    metric_r2_adj_train = round(1-(1-metric_r2_train)*(len(test)-1)/(len(test)-len(features)-1), 3)
    metric_r2_adj_test = round(1-(1-metric_r2_test)*(len(test)-1)/(len(test)-len(features)-1), 3)

    metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
    metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

    metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
    metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

    metric_train_score = round(1.00 - (metric_train_mae / train_y_real.mean()), 3)
    metric_test_score = round(1.00 - (metric_test_mae / test_y_real.mean()), 3)
    
    
    # Genere un diccionario con las métricas
    metrics = {
        "r2_train": metric_r2_train,
        "r2_test": metric_r2_test,
        "r2_adj_train": metric_r2_adj_train,
        "r2_adj_test": metric_r2_adj_test,
        "mae_train": metric_train_mae,
        "mae_test": metric_test_mae,
        "rmse_train": metric_train_rmse,
        "rmse_test": metric_test_rmse,
        "score_train": metric_train_score,
        "score_test": metric_test_score
    }
    
    
    # Loggar parámetros
    mlflow.log_params(params)
    
    # Logar métricas
    mlflow.log_metrics(metrics)
    
    # Schema input/output del modelo
    input_schema = Schema([
        ColSpec("string", "Epoca_Covid"),
        # ColSpec("string", "Trimestre"),
        ColSpec("string", "Mes"),
        ColSpec("string", "Tipo_Cliente"),
        # ColSpec("string", "Distribuidor_MSM"),
        ColSpec("string", "Area_Dist"),
        # ColSpec("string", "Zona"),
        # ColSpec("string", "ID_CCAA"),
        ColSpec("string", "ID_Provincia"),
        ColSpec("string", "Gama"),
        ColSpec("string", "Formato_1"),
        ColSpec("string", "Formato_2"),
        ColSpec("double", "Numerica"),
        ColSpec("double", "Num_PdV_CI"),
        ColSpec("double", "Num_PdV_RU"),
        ColSpec("double", "Num_PdV_TU"),
        ColSpec("double", "Impactos_TV"),
        # ColSpec("double", "Impactos_OOH"),
        # ColSpec("double", "Impresiones_Meta"),
        # ColSpec("double", "Impresiones_Pint"),
        # ColSpec("double", "Impresiones_Digi"),
        ColSpec("double", "Sell_In_PP"),
        ColSpec("double", "Marketing_Nacional"),
        ColSpec("double", "Trade_Marketing"),
        ColSpec("double", "Marketing_PLV"),
        # ColSpec("double", "Distancia_CdP"),
        ColSpec("double", "Dispersion_PdV"),
        ColSpec("double", "Volumen_Solicitado"),
        # ColSpec("double", "Nivel_Servicio"),
        ColSpec("double", "Tarifa_Eur_Hl"),
        ColSpec("double", "Margen_Distribuidor"),
        # ColSpec("double", "Num_Festivos"),
        # ColSpec("integer", "Num_Festivos_AI"),
        ColSpec("double", "Egatur"),
        ColSpec("double", "Familitur"),
        # ColSpec("double", "Movimientos_Neto"),
        # ColSpec("double", "IPC"),
        # ColSpec("double", "IPI"),
        ColSpec("double", "IPRI"),
        # ColSpec("double", "Temperatura_Media"),
        # ColSpec("double", "Precipitacion"),
        ColSpec("double", "Sell_In_1")
    ])
    
    output_schema = Schema([ColSpec("double", "prediction")])
    
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    # Log modelo entrenado
    mlflow.sklearn.log_model(model, model_name, signature=signature)

    
    
    
# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f"El tiempo de ejecución es de {tiempo_ejecucion/60} minutos")


# Info de la ejecución
print("")
print(f"Modelo loggado en el experimento: {experiment_id} --> {experiment_name}")
print(f"Modelo loggado en la ejecución: {run_id} --> {run_name}")
print(f"Nombre del modelo: {model_name}")
print(f"Modelo URI: 'runs:/{run_id}/artifacts/{model_name}'")

[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0

El tiempo de ejecución es de 2.3808906157811482 minutos

Modelo loggado en el experimento: 543627291029851046 --> MSM_Sell_In_Explicabilidad
Modelo loggado en la ejecución: c2edff31091d4fa3867e83daa09920d9 --> Modelo_XAI_LightGBM_Experiment_1
Nombre del modelo: lightgbm-model-exp-1
Modelo URI: 'runs:/c2edff31091d4fa3867e83daa09920d9/artifacts/lightgbm-model-exp-1'


/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## 3.3. LightGBM (Reducción Variables - Experimento 2)

In [24]:
# Medir tiempo
tiempo_inicial = time()


# Variables de ejecución MLflow
# ------------------------------------------------------------------------------------
experiment_name = 'MSM_Sell_In_Explicabilidad'
run_name = 'Modelo_XAI_LightGBM_Experiment_2'
model_name = 'lightgbm-model-exp-2'


# Seed
# ------------------------------------------------------------------------------------
seed = 123


# Dataset de test y entrenamiento
# ------------------------------------------------------------------------------------
# Copia del dataset
df_exp = df.copy()

# Eliminar variables con menor infulencia en el modelo (sobre 100)
df_exp = df_exp.drop(columns=[
    # Experimento 1
    'Num_Festivos_AI',      # 0.00
    'Distribuidor_MSM',     # 0.01
    'Impactos_OOH',         # 0.01
    'Impresiones_Digi',     # 0.04
    'Impresiones_Pint',     # 0.06
    'Trimestre',            # 0.09
    'Num_Festivos',         # 0.11
    'ID_CCAA',              # 0.11
    'Nivel_Servicio',       # 0.12
    'Zona',                 # 0.21
    'Precipitacion',        # 0.23
    #'Num_PdV_RU',          # 0.29
    'Temperatura_Media',    # 0.31
    #'Num_PdV_TU',          # 0.33
    'Distancia_CdP',        # 0.33
    'Movimientos_Neto',     # 0.35
    'IPI',                  # 0.41
    'Impresiones_Meta',     # 0.42
    'IPC',                  # 0.49
    # Experimento 2
    'Egatur',               # 0.54
    'Epoca_Covid',          # 0.63
    'Area_Dist',            # 0.69
])

# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df_exp.columns)
features.remove(target)

# Variables categóricas
categorical_features = [col for col in df_exp.columns if df_exp[col].dtype == 'category']

# Dataset de test y entrenamiento
train, test = train_test_split(df_exp, train_size=0.95, test_size=0.05, random_state=seed)



# MLflow
# ------------------------------------------------------------------------------------
# Activar el experimento
mlflow.set_experiment(experiment_name)

# Lanzar ejecución de MLflow
with mlflow.start_run(run_name=run_name) as run:
    
    # UUID del experimento
    experiment_id = run.info.experiment_id
    
    # UUID de la ejecución
    run_id = run.info.run_uuid
    
    # Instanciar OrdinalEncoder
    encoder = OrdinalEncoder(
        cols=categorical_features,
        handle_unknown="ignore",
        return_df=True
    )

    
    # Parámetros
    params = {
        "boosting_type": "gbdt",
        "learning_rate": 0.1,
        "n_estimators": 1000,
        "max_depth": 8,
        "num_leaves": 120,
        "min_child_samples": 10,
        "bagging_fraction": 1.0,
        "bagging_freq": 0,
        "feature_fraction": 0.8,
        "reg_sqrt": True,
        "feature_pre_filter": False,
        "random_state": seed,
        "n_jobs": -1
    }
    

    # Instanciar LGBMRegressor
    mlgb = lgb.LGBMRegressor(**params)

    
    # Pipeline: Encoder + Regresor
    model_pipeline = Pipeline([
        ("encoder", encoder),
        ("mlgb", mlgb)
    ])
    
    
    # Entrenar modelo
    model = model_pipeline.fit(train[features], train[target])
    
    
    # Cálculo de métricas
    train_y_real = train[target]
    train_y_pred = model.predict(train[features])

    test_y_real = test[target]
    test_y_pred = model.predict(test[features])

    metric_r2_train = round(r2_score(train_y_real, train_y_pred), 3)
    metric_r2_test = round(r2_score(test_y_real, test_y_pred), 3)

    metric_r2_adj_train = round(1-(1-metric_r2_train)*(len(test)-1)/(len(test)-len(features)-1), 3)
    metric_r2_adj_test = round(1-(1-metric_r2_test)*(len(test)-1)/(len(test)-len(features)-1), 3)

    metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
    metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

    metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
    metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

    metric_train_score = round(1.00 - (metric_train_mae / train_y_real.mean()), 3)
    metric_test_score = round(1.00 - (metric_test_mae / test_y_real.mean()), 3)
    
    
    # Genere un diccionario con las métricas
    metrics = {
        "r2_train": metric_r2_train,
        "r2_test": metric_r2_test,
        "r2_adj_train": metric_r2_adj_train,
        "r2_adj_test": metric_r2_adj_test,
        "mae_train": metric_train_mae,
        "mae_test": metric_test_mae,
        "rmse_train": metric_train_rmse,
        "rmse_test": metric_test_rmse,
        "score_train": metric_train_score,
        "score_test": metric_test_score
    }
    
    
    # Loggar parámetros
    mlflow.log_params(params)
    
    # Logar métricas
    mlflow.log_metrics(metrics)
    
    # Schema input/output del modelo
    input_schema = Schema([
        # ColSpec("string", "Epoca_Covid"),
        # ColSpec("string", "Trimestre"),
        ColSpec("string", "Mes"),
        ColSpec("string", "Tipo_Cliente"),
        # ColSpec("string", "Distribuidor_MSM"),
        # ColSpec("string", "Area_Dist"),
        # ColSpec("string", "Zona"),
        # ColSpec("string", "ID_CCAA"),
        ColSpec("string", "ID_Provincia"),
        ColSpec("string", "Gama"),
        ColSpec("string", "Formato_1"),
        ColSpec("string", "Formato_2"),
        ColSpec("double", "Numerica"),
        ColSpec("double", "Num_PdV_CI"),
        ColSpec("double", "Num_PdV_RU"),
        ColSpec("double", "Num_PdV_TU"),
        ColSpec("double", "Impactos_TV"),
        # ColSpec("double", "Impactos_OOH"),
        # ColSpec("double", "Impresiones_Meta"),
        # ColSpec("double", "Impresiones_Pint"),
        # ColSpec("double", "Impresiones_Digi"),
        ColSpec("double", "Sell_In_PP"),
        ColSpec("double", "Marketing_Nacional"),
        ColSpec("double", "Trade_Marketing"),
        ColSpec("double", "Marketing_PLV"),
        # ColSpec("double", "Distancia_CdP"),
        ColSpec("double", "Dispersion_PdV"),
        ColSpec("double", "Volumen_Solicitado"),
        # ColSpec("double", "Nivel_Servicio"),
        ColSpec("double", "Tarifa_Eur_Hl"),
        ColSpec("double", "Margen_Distribuidor"),
        # ColSpec("double", "Num_Festivos"),
        # ColSpec("integer", "Num_Festivos_AI"),
        # ColSpec("double", "Egatur"),
        ColSpec("double", "Familitur"),
        # ColSpec("double", "Movimientos_Neto"),
        # ColSpec("double", "IPC"),
        # ColSpec("double", "IPI"),
        ColSpec("double", "IPRI"),
        # ColSpec("double", "Temperatura_Media"),
        # ColSpec("double", "Precipitacion"),
        ColSpec("double", "Sell_In_1")
    ])
    
    output_schema = Schema([ColSpec("double", "prediction")])
    
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    # Log modelo entrenado
    mlflow.sklearn.log_model(model, model_name, signature=signature)

    
    
    
# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f"El tiempo de ejecución es de {tiempo_ejecucion/60} minutos")


# Info de la ejecución
print("")
print(f"Modelo loggado en el experimento: {experiment_id} --> {experiment_name}")
print(f"Modelo loggado en la ejecución: {run_id} --> {run_name}")
print(f"Nombre del modelo: {model_name}")
print(f"Modelo URI: 'runs:/{run_id}/artifacts/{model_name}'")

[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0

El tiempo de ejecución es de 2.340587917963664 minutos

Modelo loggado en el experimento: 543627291029851046 --> MSM_Sell_In_Explicabilidad
Modelo loggado en la ejecución: 0358ec38eed14c69ac7e8ebd6590ebc2 --> Modelo_XAI_LightGBM_Experiment_2
Nombre del modelo: lightgbm-model-exp-2
Modelo URI: 'runs:/0358ec38eed14c69ac7e8ebd6590ebc2/artifacts/lightgbm-model-exp-2'


/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## 3.4. LightGBM (Reducción Variables - Experimento 3)

In [25]:
# Medir tiempo
tiempo_inicial = time()


# Variables de ejecución MLflow
# ------------------------------------------------------------------------------------
experiment_name = 'MSM_Sell_In_Explicabilidad'
run_name = 'Modelo_XAI_LightGBM_Experiment_3'
model_name = 'lightgbm-model-exp-3'


# Seed
# ------------------------------------------------------------------------------------
seed = 123


# Dataset de test y entrenamiento
# ------------------------------------------------------------------------------------
# Copia del dataset
df_exp = df.copy()

# Eliminar variables con menor infulencia en el modelo (sobre 100)
df_exp = df_exp.drop(columns=[
    # Experimento 1
    'Num_Festivos_AI',      # 0.00
    'Distribuidor_MSM',     # 0.01
    'Impactos_OOH',         # 0.01
    'Impresiones_Digi',     # 0.04
    'Impresiones_Pint',     # 0.06
    'Trimestre',            # 0.09
    'Num_Festivos',         # 0.11
    'ID_CCAA',              # 0.11
    'Nivel_Servicio',       # 0.12
    'Zona',                 # 0.21
    'Precipitacion',        # 0.23
    #'Num_PdV_RU',          # 0.29
    'Temperatura_Media',    # 0.31
    #'Num_PdV_TU',          # 0.33
    'Distancia_CdP',        # 0.33
    'Movimientos_Neto',     # 0.35
    'IPI',                  # 0.41
    'Impresiones_Meta',     # 0.42
    'IPC',                  # 0.49
    # Experimento 2
    'Egatur',               # 0.54
    'Epoca_Covid',          # 0.63
    'Area_Dist',            # 0.69
    # Experimento 3
    #'Num_PdV_CI',          # 0.71
    'Impactos_TV',          # 0.80
    'Dispersion_PdV'        # 0.82
])

# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df_exp.columns)
features.remove(target)

# Variables categóricas
categorical_features = [col for col in df_exp.columns if df_exp[col].dtype == 'category']

# Dataset de test y entrenamiento
train, test = train_test_split(df_exp, train_size=0.95, test_size=0.05, random_state=seed)



# MLflow
# ------------------------------------------------------------------------------------
# Activar el experimento
mlflow.set_experiment(experiment_name)

# Lanzar ejecución de MLflow
with mlflow.start_run(run_name=run_name) as run:
    
    # UUID del experimento
    experiment_id = run.info.experiment_id
    
    # UUID de la ejecución
    run_id = run.info.run_uuid
    
    # Instanciar OrdinalEncoder
    encoder = OrdinalEncoder(
        cols=categorical_features,
        handle_unknown="ignore",
        return_df=True
    )

    
    # Parámetros
    params = {
        "boosting_type": "gbdt",
        "learning_rate": 0.1,
        "n_estimators": 1000,
        "max_depth": 8,
        "num_leaves": 120,
        "min_child_samples": 10,
        "bagging_fraction": 1.0,
        "bagging_freq": 0,
        "feature_fraction": 0.8,
        "reg_sqrt": True,
        "feature_pre_filter": False,
        "random_state": seed,
        "n_jobs": -1
    }
    

    # Instanciar LGBMRegressor
    mlgb = lgb.LGBMRegressor(**params)

    
    # Pipeline: Encoder + Regresor
    model_pipeline = Pipeline([
        ("encoder", encoder),
        ("mlgb", mlgb)
    ])
    
    
    # Entrenar modelo
    model = model_pipeline.fit(train[features], train[target])
    
    
    # Cálculo de métricas
    train_y_real = train[target]
    train_y_pred = model.predict(train[features])

    test_y_real = test[target]
    test_y_pred = model.predict(test[features])

    metric_r2_train = round(r2_score(train_y_real, train_y_pred), 3)
    metric_r2_test = round(r2_score(test_y_real, test_y_pred), 3)

    metric_r2_adj_train = round(1-(1-metric_r2_train)*(len(test)-1)/(len(test)-len(features)-1), 3)
    metric_r2_adj_test = round(1-(1-metric_r2_test)*(len(test)-1)/(len(test)-len(features)-1), 3)

    metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
    metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

    metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
    metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

    metric_train_score = round(1.00 - (metric_train_mae / train_y_real.mean()), 3)
    metric_test_score = round(1.00 - (metric_test_mae / test_y_real.mean()), 3)
    
    
    # Genere un diccionario con las métricas
    metrics = {
        "r2_train": metric_r2_train,
        "r2_test": metric_r2_test,
        "r2_adj_train": metric_r2_adj_train,
        "r2_adj_test": metric_r2_adj_test,
        "mae_train": metric_train_mae,
        "mae_test": metric_test_mae,
        "rmse_train": metric_train_rmse,
        "rmse_test": metric_test_rmse,
        "score_train": metric_train_score,
        "score_test": metric_test_score
    }
    
    
    # Loggar parámetros
    mlflow.log_params(params)
    
    # Logar métricas
    mlflow.log_metrics(metrics)
    
    # Schema input/output del modelo
    input_schema = Schema([
        # ColSpec("string", "Epoca_Covid"),
        # ColSpec("string", "Trimestre"),
        ColSpec("string", "Mes"),
        ColSpec("string", "Tipo_Cliente"),
        # ColSpec("string", "Distribuidor_MSM"),
        # ColSpec("string", "Area_Dist"),
        # ColSpec("string", "Zona"),
        # ColSpec("string", "ID_CCAA"),
        ColSpec("string", "ID_Provincia"),
        ColSpec("string", "Gama"),
        ColSpec("string", "Formato_1"),
        ColSpec("string", "Formato_2"),
        ColSpec("double", "Numerica"),
        ColSpec("double", "Num_PdV_CI"),
        ColSpec("double", "Num_PdV_RU"),
        ColSpec("double", "Num_PdV_TU"),
        # ColSpec("double", "Impactos_TV"),
        # ColSpec("double", "Impactos_OOH"),
        # ColSpec("double", "Impresiones_Meta"),
        # ColSpec("double", "Impresiones_Pint"),
        # ColSpec("double", "Impresiones_Digi"),
        ColSpec("double", "Sell_In_PP"),
        ColSpec("double", "Marketing_Nacional"),
        ColSpec("double", "Trade_Marketing"),
        ColSpec("double", "Marketing_PLV"),
        # ColSpec("double", "Distancia_CdP"),
        # ColSpec("double", "Dispersion_PdV"),
        ColSpec("double", "Volumen_Solicitado"),
        # ColSpec("double", "Nivel_Servicio"),
        ColSpec("double", "Tarifa_Eur_Hl"),
        ColSpec("double", "Margen_Distribuidor"),
        # ColSpec("double", "Num_Festivos"),
        # ColSpec("integer", "Num_Festivos_AI"),
        # ColSpec("double", "Egatur"),
        ColSpec("double", "Familitur"),
        # ColSpec("double", "Movimientos_Neto"),
        # ColSpec("double", "IPC"),
        # ColSpec("double", "IPI"),
        ColSpec("double", "IPRI"),
        # ColSpec("double", "Temperatura_Media"),
        # ColSpec("double", "Precipitacion"),
        ColSpec("double", "Sell_In_1")
    ])
    
    output_schema = Schema([ColSpec("double", "prediction")])
    
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    # Log modelo entrenado
    mlflow.sklearn.log_model(model, model_name, signature=signature)

    
    
    
# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f"El tiempo de ejecución es de {tiempo_ejecucion/60} minutos")


# Info de la ejecución
print("")
print(f"Modelo loggado en el experimento: {experiment_id} --> {experiment_name}")
print(f"Modelo loggado en la ejecución: {run_id} --> {run_name}")
print(f"Nombre del modelo: {model_name}")
print(f"Modelo URI: 'runs:/{run_id}/artifacts/{model_name}'")

[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0

El tiempo de ejecución es de 2.0535769979159038 minutos

Modelo loggado en el experimento: 543627291029851046 --> MSM_Sell_In_Explicabilidad
Modelo loggado en la ejecución: d55afbdaf24e446f8c2b3d314a07a00c --> Modelo_XAI_LightGBM_Experiment_3
Nombre del modelo: lightgbm-model-exp-3
Modelo URI: 'runs:/d55afbdaf24e446f8c2b3d314a07a00c/artifacts/lightgbm-model-exp-3'


/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## 3.5. LightGBM (Reducción Variables - Experimento 4)

In [26]:
# Medir tiempo
tiempo_inicial = time()


# Variables de ejecución MLflow
# ------------------------------------------------------------------------------------
experiment_name = 'MSM_Sell_In_Explicabilidad'
run_name = 'Modelo_XAI_LightGBM_Experiment_4'
model_name = 'lightgbm-model-exp-4'


# Seed
# ------------------------------------------------------------------------------------
seed = 123


# Dataset de test y entrenamiento
# ------------------------------------------------------------------------------------
# Copia del dataset
df_exp = df.copy()

# Eliminar variables con menor infulencia en el modelo (sobre 100)
df_exp = df_exp.drop(columns=[
    # Experimento 1
    'Num_Festivos_AI',      # 0.00
    'Distribuidor_MSM',     # 0.01
    'Impactos_OOH',         # 0.01
    'Impresiones_Digi',     # 0.04
    'Impresiones_Pint',     # 0.06
    'Trimestre',            # 0.09
    'Num_Festivos',         # 0.11
    'ID_CCAA',              # 0.11
    'Nivel_Servicio',       # 0.12
    'Zona',                 # 0.21
    'Precipitacion',        # 0.23
    'Num_PdV_RU',           # 0.29
    'Temperatura_Media',    # 0.31
    'Num_PdV_TU',           # 0.33
    'Distancia_CdP',        # 0.33
    'Movimientos_Neto',     # 0.35
    'IPI',                  # 0.41
    'Impresiones_Meta',     # 0.42
    'IPC',                  # 0.49
    # Experimento 4
    'Num_PdV_CI'            # 0.71
])

# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df_exp.columns)
features.remove(target)

# Variables categóricas
categorical_features = [col for col in df_exp.columns if df_exp[col].dtype == 'category']

# Dataset de test y entrenamiento
train, test = train_test_split(df_exp, train_size=0.95, test_size=0.05, random_state=seed)



# MLflow
# ------------------------------------------------------------------------------------
# Activar el experimento
mlflow.set_experiment(experiment_name)

# Lanzar ejecución de MLflow
with mlflow.start_run(run_name=run_name) as run:
    
    # UUID del experimento
    experiment_id = run.info.experiment_id
    
    # UUID de la ejecución
    run_id = run.info.run_uuid
    
    # Instanciar OrdinalEncoder
    encoder = OrdinalEncoder(
        cols=categorical_features,
        handle_unknown="ignore",
        return_df=True
    )

    
    # Parámetros
    params = {
        "boosting_type": "gbdt",
        "learning_rate": 0.1,
        "n_estimators": 1000,
        "max_depth": 8,
        "num_leaves": 120,
        "min_child_samples": 10,
        "bagging_fraction": 1.0,
        "bagging_freq": 0,
        "feature_fraction": 0.8,
        "reg_sqrt": True,
        "feature_pre_filter": False,
        "random_state": seed,
        "n_jobs": -1
    }
    

    # Instanciar LGBMRegressor
    mlgb = lgb.LGBMRegressor(**params)

    
    # Pipeline: Encoder + Regresor
    model_pipeline = Pipeline([
        ("encoder", encoder),
        ("mlgb", mlgb)
    ])
    
    
    # Entrenar modelo
    model = model_pipeline.fit(train[features], train[target])
    
    
    # Cálculo de métricas
    train_y_real = train[target]
    train_y_pred = model.predict(train[features])

    test_y_real = test[target]
    test_y_pred = model.predict(test[features])

    metric_r2_train = round(r2_score(train_y_real, train_y_pred), 3)
    metric_r2_test = round(r2_score(test_y_real, test_y_pred), 3)

    metric_r2_adj_train = round(1-(1-metric_r2_train)*(len(test)-1)/(len(test)-len(features)-1), 3)
    metric_r2_adj_test = round(1-(1-metric_r2_test)*(len(test)-1)/(len(test)-len(features)-1), 3)

    metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
    metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

    metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
    metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

    metric_train_score = round(1.00 - (metric_train_mae / train_y_real.mean()), 3)
    metric_test_score = round(1.00 - (metric_test_mae / test_y_real.mean()), 3)
    
    
    # Genere un diccionario con las métricas
    metrics = {
        "r2_train": metric_r2_train,
        "r2_test": metric_r2_test,
        "r2_adj_train": metric_r2_adj_train,
        "r2_adj_test": metric_r2_adj_test,
        "mae_train": metric_train_mae,
        "mae_test": metric_test_mae,
        "rmse_train": metric_train_rmse,
        "rmse_test": metric_test_rmse,
        "score_train": metric_train_score,
        "score_test": metric_test_score
    }
    
    
    # Loggar parámetros
    mlflow.log_params(params)
    
    # Logar métricas
    mlflow.log_metrics(metrics)
    
    # Schema input/output del modelo
    input_schema = Schema([
        ColSpec("string", "Epoca_Covid"),
        # ColSpec("string", "Trimestre"),
        ColSpec("string", "Mes"),
        ColSpec("string", "Tipo_Cliente"),
        # ColSpec("string", "Distribuidor_MSM"),
        ColSpec("string", "Area_Dist"),
        # ColSpec("string", "Zona"),
        # ColSpec("string", "ID_CCAA"),
        ColSpec("string", "ID_Provincia"),
        ColSpec("string", "Gama"),
        ColSpec("string", "Formato_1"),
        ColSpec("string", "Formato_2"),
        ColSpec("double", "Numerica"),
        # ColSpec("double", "Num_PdV_CI"),
        # ColSpec("double", "Num_PdV_RU"),
        # ColSpec("double", "Num_PdV_TU"),
        ColSpec("double", "Impactos_TV"),
        # ColSpec("double", "Impactos_OOH"),
        # ColSpec("double", "Impresiones_Meta"),
        # ColSpec("double", "Impresiones_Pint"),
        # ColSpec("double", "Impresiones_Digi"),
        ColSpec("double", "Sell_In_PP"),
        ColSpec("double", "Marketing_Nacional"),
        ColSpec("double", "Trade_Marketing"),
        ColSpec("double", "Marketing_PLV"),
        # ColSpec("double", "Distancia_CdP"),
        ColSpec("double", "Dispersion_PdV"),
        ColSpec("double", "Volumen_Solicitado"),
        # ColSpec("double", "Nivel_Servicio"),
        ColSpec("double", "Tarifa_Eur_Hl"),
        ColSpec("double", "Margen_Distribuidor"),
        # ColSpec("double", "Num_Festivos"),
        # ColSpec("integer", "Num_Festivos_AI"),
        ColSpec("double", "Egatur"),
        ColSpec("double", "Familitur"),
        # ColSpec("double", "Movimientos_Neto"),
        # ColSpec("double", "IPC"),
        # ColSpec("double", "IPI"),
        ColSpec("double", "IPRI"),
        # ColSpec("double", "Temperatura_Media"),
        # ColSpec("double", "Precipitacion"),
        ColSpec("double", "Sell_In_1")
    ])
    
    output_schema = Schema([ColSpec("double", "prediction")])
    
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    # Log modelo entrenado
    mlflow.sklearn.log_model(model, model_name, signature=signature)

    
    
    
# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f"El tiempo de ejecución es de {tiempo_ejecucion/60} minutos")


# Info de la ejecución
print("")
print(f"Modelo loggado en el experimento: {experiment_id} --> {experiment_name}")
print(f"Modelo loggado en la ejecución: {run_id} --> {run_name}")
print(f"Nombre del modelo: {model_name}")
print(f"Modelo URI: 'runs:/{run_id}/artifacts/{model_name}'")

[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0

El tiempo de ejecución es de 2.2411839683850605 minutos

Modelo loggado en el experimento: 543627291029851046 --> MSM_Sell_In_Explicabilidad
Modelo loggado en la ejecución: 0e600fb121e841f6a9da21236e2fe3f2 --> Modelo_XAI_LightGBM_Experiment_4
Nombre del modelo: lightgbm-model-exp-4
Modelo URI: 'runs:/0e600fb121e841f6a9da21236e2fe3f2/artifacts/lightgbm-model-exp-4'


/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## 3.6. XGBoost (Todas las variables)

In [44]:
# Medir tiempo
tiempo_inicial = time()




# MLflow
# ------------------------------------------------------------------------------------
# Activar el experimento
mlflow.set_experiment(name)

# Establecer nombre de la ejecuión de MLflow
run_name = "Modelo_XAI_XGBoost"

# Establecer nombre del modelo
model_name = "xgboost-model"

# Lanzar ejecución de MLflow
with mlflow.start_run(run_name=run_name) as run:
    
    # UUID del experimento
    experiment_id = run.info.experiment_id
    
    # UUID de la ejecución
    run_id = run.info.run_uuid
    
    # Instanciar OrdinalEncoder
    encoder = OrdinalEncoder(
        cols=categorical_features,
        handle_unknown='ignore',
        return_df=True
    )

    
    # Parámetros
    params = {
        'booster': 'gbtree',
        'learning_rate': 0.1,
        'n_estimators': 1000,
        'max_depth': 8,
        'num_leaves': 80,
        'colsample_bytree': 0.9,
        'subsample': 0.9,
        'random_state': seed,
        'n_jobs': -1,
        'verbosity': 1
    }
    

    # Instanciar LGBMRegressor
    xgbr = XGBRegressor(**params)

    
    # Pipeline: Encoder + Regresor
    model_pipeline = Pipeline([
        ("encoder", encoder),
        ("xgbr", xgbr)
        ])
    
    
    # Entrenar modelo
    model = model_pipeline.fit(train[features], train[target])
    
    
    # Cálculo de métricas
    train_y_real = train[target]
    train_y_pred = model.predict(train[features])

    test_y_real = test[target]
    test_y_pred = model.predict(test[features])

    metric_r2_train = round(r2_score(train_y_real, train_y_pred), 3)
    metric_r2_test = round(r2_score(test_y_real, test_y_pred), 3)

    metric_r2_adj_train = round(1-(1-metric_r2_train)*(len(test)-1)/(len(test)-len(features)-1), 3)
    metric_r2_adj_test = round(1-(1-metric_r2_test)*(len(test)-1)/(len(test)-len(features)-1), 3)

    metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
    metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

    metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
    metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

    metric_train_score = round(1.00 - (metric_train_mae / train_y_real.mean()), 3)
    metric_test_score = round(1.00 - (metric_test_mae / test_y_real.mean()), 3)
    
    
    # Genere un diccionario con las métricas
    metrics = {
        "r2_train": metric_r2_train,
        "r2_test": metric_r2_test,
        "r2_adj_train": metric_r2_adj_train,
        "r2_adj_test": metric_r2_adj_test,
        "mae_train": metric_train_mae,
        "mae_test": metric_test_mae,
        "rmse_train": metric_train_rmse,
        "rmse_test": metric_test_rmse,
        "score_train": metric_train_score,
        "score_test": metric_test_score
    }
    
    
    # Loggar parámetros
    mlflow.log_params(params)
  
    # Logar métricas
    mlflow.log_metrics(metrics)
    
    # Schema input/output del modelo
    input_schema = Schema([
        ColSpec("string", "Epoca_Covid"),
        ColSpec("string", "Trimestre"),
        ColSpec("string", "Mes"),
        ColSpec("string", "Tipo_Cliente"),
        ColSpec("string", "Distribuidor_MSM"),
        ColSpec("string", "Area_Dist"),
        ColSpec("string", "Zona"),
        ColSpec("string", "ID_CCAA"),
        ColSpec("string", "ID_Provincia"),
        ColSpec("string", "Gama"),
        ColSpec("string", "Formato"),
        ColSpec("double", "Num_PdV_CI"),
        ColSpec("double", "Num_PdV_RU"),
        ColSpec("double", "Num_PdV_TU"),
        ColSpec("double", "Impactos_TV"),
        ColSpec("double", "Impactos_OOH"),
        ColSpec("double", "Impresiones_Meta"),
        ColSpec("double", "Impresiones_Pint"),
        ColSpec("double", "Impresiones_Digi"),
        ColSpec("double", "Sell_In_PP"),
        ColSpec("double", "Marketing_Nacional"),
        ColSpec("double", "Trade_Marketing"),
        ColSpec("double", "Marketing_PLV"),
        ColSpec("double", "Distancia_CdP"),
        ColSpec("double", "Dispersion_PdV"),
        ColSpec("double", "Volumen_Solicitado"),
        ColSpec("double", "Nivel_Servicio"),
        ColSpec("double", "Tarifa_Eur_Hl"),
        ColSpec("double", "Margen_Distribuidor"),
        ColSpec("double", "Num_Festivos"),
        ColSpec("long", "Num_Festivos_AI"),
        ColSpec("double", "Egatur"),
        ColSpec("double", "Familitur"),
        ColSpec("double", "Movimientos_Neto"),
        ColSpec("double", "IPC"),
        ColSpec("double", "IPI"),
        ColSpec("double", "IPRI"),
        ColSpec("double", "Temperatura_Media"),
        ColSpec("double", "Precipitacion"),
        ColSpec("double", "Sell_In_1")
    ])
    
    output_schema = Schema([ColSpec("double", "prediction")])
    
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    # Log modelo entrenado
    mlflow.sklearn.log_model(model, model_name, signature=signature)

    
    
    
# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f"El tiempo de ejecución es de {tiempo_ejecucion/60} minutos")


# Info de la ejecución
print("")
print(f"Modelo loggado en el experimento: {experiment_id} --> {experiment_name}")
print(f"Modelo loggado en la ejecución: {run_id} --> {run_name}")
print(f"Nombre del modelo: {model_name}")
print(f"Modelo URI: 'runs:/{run_id}/artifacts/{model_name}'")

[13:03:02] WARNING: ../src/learner.cc:767: 
Parameters: { "num_leaves" } are not used.


Modelo loggado en el experimento: 992663646765353795 --> MSM_Sell_In_Explicabilidad
Modelo loggado en la ejecución: 3a2bf0965b164c208fe5a69291c65bd1 --> Modelo_XAI_XGBoost

El tiempo de ejecución es de 33.51186156670253 minutos


/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/gmachin/.cache/pypoetry/virtualenvs/msm-sell-in-prediction-unVKpS6j-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# 4. Predicciones

In [45]:
test[:1]

,Epoca_Covid,Trimestre,Mes,Tipo_Cliente,Distribuidor_MSM,Area_Dist,Zona,ID_CCAA,ID_Provincia,Gama,...,Egatur,Familitur,Movimientos_Neto,IPC,IPI,IPRI,Temperatura_Media,Precipitacion,Sell_In_1,Sell_In
648740,pre-covid,T1,1,DHC,0,ZZ,ZM8,13,28,SM,...,644.099976,0.0,-465109.25,-0.8,96.830002,102.110001,6.47,52.060001,0.24,0.4


In [46]:
test[features][:1]

,Epoca_Covid,Trimestre,Mes,Tipo_Cliente,Distribuidor_MSM,Area_Dist,Zona,ID_CCAA,ID_Provincia,Gama,...,Num_Festivos_AI,Egatur,Familitur,Movimientos_Neto,IPC,IPI,IPRI,Temperatura_Media,Precipitacion,Sell_In_1
648740,pre-covid,T1,1,DHC,0,ZZ,ZM8,13,28,SM,...,0,644.099976,0.0,-465109.25,-0.8,96.830002,102.110001,6.47,52.060001,0.24


In [47]:
df_to_pred = test[features][:1]

categorical_columns = df_to_pred.select_dtypes(include=['category']).columns
df_to_pred[categorical_columns] = df_to_pred[categorical_columns].astype('object')

df_to_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 648740 to 648740
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Epoca_Covid          1 non-null      object 
 1   Trimestre            1 non-null      object 
 2   Mes                  1 non-null      object 
 3   Tipo_Cliente         1 non-null      object 
 4   Distribuidor_MSM     1 non-null      object 
 5   Area_Dist            1 non-null      object 
 6   Zona                 1 non-null      object 
 7   ID_CCAA              1 non-null      object 
 8   ID_Provincia         1 non-null      object 
 9   Gama                 1 non-null      object 
 10  Formato              1 non-null      object 
 11  Num_PdV_CI           1 non-null      float32
 12  Num_PdV_RU           1 non-null      float32
 13  Num_PdV_TU           1 non-null      float32
 14  Impactos_TV          1 non-null      float64
 15  Impactos_OOH         1 non-null   

In [48]:
# Instanciar modelo loggado
logged_model_lgb = 'runs:/06e49b53f85e4761a3ebc22ae1d16933/lightgbm-model'

# Cargar modelo como PyFuncModel.
loaded_model_lgb = mlflow.pyfunc.load_model(logged_model_lgb)

# Prediccion
loaded_model_lgb.predict(df_to_pred[features][:1])

array([0.31627984])

In [51]:
# Instanciar modelo loggado
logged_model_xgb = 'runs:/3a2bf0965b164c208fe5a69291c65bd1/xgboost_model'

# Cargar modelo como PyFuncModel.
loaded_model_xgb = mlflow.pyfunc.load_model(logged_model_xgb)

# Prediccion
loaded_model_xgb.predict(df_to_pred[features][:1])

array([0.23122253], dtype=float32)

In [ ]:
https://github.com/mtpatter/mlflow-tutorial/blob/main/mlflow-sklearn.ipynb